# Volatility Indicators - Apple Stock 2023-2025

Testing volatility indicators: Bollinger Bands, ATR, ADR, APZ, SAR.

In [2]:
import sys
from pathlib import Path

import plotly.graph_objects as go
import yfinance as yf
from plotly.subplots import make_subplots

sys.path.insert(0, str(Path("../..").resolve()))

from indicators.volatility import calculate_adr, calculate_apz, calculate_atr, calculate_bbands, calculate_sar

In [3]:
# Fetch Apple data
df = yf.download("AAPL", start="2023-01-01", end="2025-10-01", auto_adjust=True, progress=False)

# Handle MultiIndex columns from yfinance
if df.columns.nlevels == 2:
    df.columns = df.columns.get_level_values(0)

df.columns = df.columns.str.lower()
df = df.reset_index()
df.columns = df.columns.str.lower()

print(f"Data shape: {df.shape}")
df.head()

Data shape: (688, 6)


Price,date,close,high,low,open,volume
0,2023-01-03,123.330650,129.079567,122.443165,128.468194,112117500
1,2023-01-04,124.602722,126.870739,123.340524,125.125350,89113600
2,2023-01-05,123.281349,125.993105,123.024970,125.362006,80962700
3,2023-01-06,127.817398,128.478079,123.153181,124.257608,87754700
4,2023-01-09,128.340012,131.554685,128.083633,128.655569,70790800


## 1. Bollinger Bands

Volatility bands = SMA ± (2 × standard deviation). Bands widen with volatility, narrow with consolidation. Price touching bands signals extremes.

In [4]:
bb_upper, bb_middle, bb_lower = calculate_bbands(df, period=20, nbdevup=2.0, nbdevdn=2.0)
df["bb_upper"] = bb_upper
df["bb_middle"] = bb_middle
df["bb_lower"] = bb_lower

fig = go.Figure()

# Bands
fig.add_trace(
    go.Scatter(x=df["date"], y=df["bb_upper"], name="Upper Band", line={"color": "gray", "dash": "dash"}, opacity=0.5)
)
fig.add_trace(go.Scatter(x=df["date"], y=df["bb_middle"], name="Middle (SMA 20)", line={"color": "blue"}))
fig.add_trace(
    go.Scatter(
        x=df["date"],
        y=df["bb_lower"],
        name="Lower Band",
        line={"color": "gray", "dash": "dash"},
        opacity=0.5,
        fill="tonexty",
        fillcolor="rgba(200,200,200,0.2)",
    )
)

# Price
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 2}))

fig.update_layout(height=600, title_text="Bollinger Bands (20, 2)", xaxis_title="Date", yaxis_title="Price")
fig.show()

print("Bollinger Bands = SMA ± (2 * StdDev)")
print(f"Price touches upper band: {(df['close'] >= df['bb_upper']).sum()} times")
print(f"Price touches lower band: {(df['close'] <= df['bb_lower']).sum()} times")

Bollinger Bands = SMA ± (2 * StdDev)
Price touches upper band: 43 times
Price touches lower band: 30 times


## 2. ATR - Average True Range

Measures average volatility over N periods. Higher ATR = higher volatility. Used for position sizing and stop-loss placement.

In [5]:
df["atr"] = calculate_atr(df, period=14)

fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=("Price", "ATR (14) - Volatility Measure")
)

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black"}), row=1, col=1)

fig.add_trace(go.Scatter(x=df["date"], y=df["atr"], name="ATR", fill="tozeroy", line={"color": "orange"}), row=2, col=1)

fig.update_layout(height=600, title_text="Average True Range")
fig.show()

print(f"ATR mean: ${df['atr'].mean():.2f}")
print(f"ATR max: ${df['atr'].max():.2f}")
print("Higher ATR = Higher volatility")

ATR mean: $4.20
ATR max: $11.57
Higher ATR = Higher volatility


## 3. ADR - Average Daily Range

Average daily range as percentage: ADR = 100 × (SMA(high/low) - 1). Measures typical daily volatility in percentage terms.

In [6]:
df["adr"] = calculate_adr(df, length=20)

fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=("Price", "ADR (20) - Daily Range %")
)

fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black"}), row=1, col=1)

fig.add_trace(
    go.Scatter(x=df["date"], y=df["adr"], name="ADR %", fill="tozeroy", line={"color": "purple"}), row=2, col=1
)

fig.update_layout(height=600, title_text="Average Daily Range")
fig.show()

print(f"ADR mean: {df['adr'].mean():.2f}%")
print(f"ADR range: {df['adr'].min():.2f}% to {df['adr'].max():.2f}%")

ADR mean: 1.94%
ADR range: 1.14% to 5.09%


## 4. APZ - Adaptive Price Zone

Adaptive volatility bands using double-smoothed EMA. More adaptive than Bollinger Bands. Adjusts to changing market conditions.

In [7]:
apz_upper, apz_lower = calculate_apz(df, period=21, band_pct=2.0)
df["apz_upper"] = apz_upper
df["apz_lower"] = apz_lower

fig = go.Figure()

# APZ bands
fig.add_trace(go.Scatter(x=df["date"], y=df["apz_upper"], name="APZ Upper", line={"color": "red", "dash": "dash"}))
fig.add_trace(
    go.Scatter(
        x=df["date"],
        y=df["apz_lower"],
        name="APZ Lower",
        line={"color": "green", "dash": "dash"},
        fill="tonexty",
        fillcolor="rgba(100,200,100,0.1)",
    )
)

# Price
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 2}))

fig.update_layout(height=600, title_text="Adaptive Price Zone (21, 2.0)", xaxis_title="Date", yaxis_title="Price")
fig.show()

print("APZ uses double-smoothed EMA for adaptive volatility bands")

APZ uses double-smoothed EMA for adaptive volatility bands


## 5. SAR - Parabolic SAR

Parabolic stop and reverse trailing stop. SAR below price = uptrend, above price = downtrend. SAR switches sides = trend reversal.

In [8]:
df["sar"] = calculate_sar(df, acceleration=0.02, maximum=0.2)

fig = go.Figure()

# Price
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 2}))

# SAR dots
fig.add_trace(go.Scatter(x=df["date"], y=df["sar"], name="SAR", mode="markers", marker={"color": "red", "size": 3}))

fig.update_layout(height=600, title_text="Parabolic SAR (Stop and Reverse)", xaxis_title="Date", yaxis_title="Price")
fig.show()

print("SAR tracks price with trailing stop")
print("SAR below price = uptrend")
print("SAR above price = downtrend")
print("SAR switches sides = trend reversal")

SAR tracks price with trailing stop
SAR below price = uptrend
SAR above price = downtrend
SAR switches sides = trend reversal


## Bollinger Bands vs APZ Comparison

In [9]:
fig = go.Figure()

# Price
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 2}))

# Bollinger Bands
fig.add_trace(
    go.Scatter(x=df["date"], y=df["bb_upper"], name="BB Upper", line={"color": "blue", "dash": "dash"}, opacity=0.7)
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["bb_lower"], name="BB Lower", line={"color": "blue", "dash": "dash"}, opacity=0.7)
)

# APZ
fig.add_trace(
    go.Scatter(x=df["date"], y=df["apz_upper"], name="APZ Upper", line={"color": "red", "dash": "dot"}, opacity=0.7)
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["apz_lower"], name="APZ Lower", line={"color": "green", "dash": "dot"}, opacity=0.7)
)

fig.update_layout(height=600, title_text="Bollinger Bands vs APZ Comparison", xaxis_title="Date", yaxis_title="Price")
fig.show()

print("BB = Standard deviation based")
print("APZ = Double-smoothed EMA based (more adaptive)")

BB = Standard deviation based
APZ = Double-smoothed EMA based (more adaptive)


## Combined View - All Volatility Indicators

In [ ]:
fig = make_subplots(
    rows=4,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    subplot_titles=(
        "Price with Bollinger Bands & SAR",
        "ATR - Absolute Volatility",
        "ADR - Relative Daily Range %",
        "APZ Bands",
    ),
)

# Row 1: Price with BB and SAR
fig.add_trace(
    go.Scatter(x=df["date"], y=df["bb_upper"], name="BB Upper", line={"color": "gray", "dash": "dash"}, opacity=0.5),
    row=1,
    col=1,
)
fig.add_trace(go.Scatter(x=df["date"], y=df["bb_middle"], name="BB Middle", line={"color": "blue"}), row=1, col=1)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["bb_lower"], name="BB Lower", line={"color": "gray", "dash": "dash"}, opacity=0.5),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["close"], name="Close", line={"color": "black", "width": 1.5}), row=1, col=1
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["sar"], name="SAR", mode="markers", marker={"color": "red", "size": 2}), row=1, col=1
)

# Row 2: ATR
fig.add_trace(go.Scatter(x=df["date"], y=df["atr"], name="ATR", fill="tozeroy", line={"color": "orange"}), row=2, col=1)

# Row 3: ADR
fig.add_trace(
    go.Scatter(x=df["date"], y=df["adr"], name="ADR %", fill="tozeroy", line={"color": "purple"}), row=3, col=1
)

# Row 4: APZ
fig.add_trace(
    go.Scatter(x=df["date"], y=df["apz_upper"], name="APZ Upper", line={"color": "red", "dash": "dash"}), row=4, col=1
)
fig.add_trace(
    go.Scatter(x=df["date"], y=df["apz_lower"], name="APZ Lower", line={"color": "green", "dash": "dash"}), row=4, col=1
)
fig.add_trace(go.Scatter(x=df["date"], y=df["close"], name="Price", line={"color": "black", "width": 1}), row=4, col=1)

fig.update_layout(height=1200, title_text="Volatility Indicators - Complete Overview")
fig.show()

print("\n=== Volatility Summary ===")
print(f"ATR mean: ${df['atr'].mean():.2f}")
print(f"ADR mean: {df['adr'].mean():.2f}%")
print(
    f"BB band touches: Upper={((df['close'] >= df['bb_upper']).sum())}, Lower={(df['close'] <= df['bb_lower']).sum()}"
)
print("\nVolatility indicators help identify risk and potential reversals.")
print("High volatility often precedes trend changes.")